In [1]:
!pip install requests

In [2]:
client_id = '3365422ef607400b8a828303186993aa'
client_secret = 'ab74afcd7a844ac7b80b8dfcbfebb495'

In [18]:
import base64
import requests
import datetime
import pandas as pd
import numpy as np
from urllib.parse import urlencode

In [4]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = None
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_token_header(self):
        if self.client_id == None or self.client_secret == None:
            raise Exception('Invalid client ID/ client secret')
        authorization = base64.b64encode((f'{self.client_id}:{self.client_secret}').encode())
        header = {
            'Authorization': f'Basic {authorization.decode()}'
        }
        return header
    
    def get_token_data(self):
        request = {
            'grant_type': 'client_credentials'
        }
        return request
        
    def get_access_token(self):
        r = requests.post(self.token_url, data=self.get_token_data(), headers=self.get_token_header())
        if r.status_code not in range(200,299):
            return False
        data = r.json()
        expires_in = data['expires_in'] # seconds
        self.access_token_expires = datetime.datetime.now() + datetime.timedelta(seconds=expires_in)
        self.access_token = data['access_token']
        return True
    
    def get_resource_header(self):
        auth = self.get_access_token()
        if auth:
            headers = {
                "Authorization": f"Bearer {self.access_token}"
            }
            return headers
        return {}
    
    def get_resource(self, href):
        endpoint = href
        header = self.get_resource_header()
        print('Token expires:', client.access_token_expires.strftime("%m/%d/%Y, %H:%M:%S"))
        r = requests.get(endpoint, headers=header)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def search(self, endpoint, query, search_type, limit = '20', offset = '0'):
        header = self.get_resource_header()
        print('Token expires:', client.access_token_expires.strftime("%m/%d/%Y, %H:%M:%S"))
        data = urlencode({
            'q': query,
            'type': search_type.lower(),
            'limit': limit,
            'offset': offset
        })
        lookup_url = f'{endpoint}?{data}'
        r = requests.get(lookup_url, headers = header)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_audio_features(self, ids):
        header = self.get_resource_header()
        endpoint = 'https://api.spotify.com/v1/audio-features'
        print('Token expires:', client.access_token_expires.strftime("%m/%d/%Y, %H:%M:%S"))
        data = urlencode({
            'ids': ids
        }).replace("%5B%27", "").replace("%27%2C+%27", ",").replace("%27%5D", "")
        lookup_url = f'{endpoint}?{data}'
        r = requests.get(lookup_url, headers = header)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

In [5]:
client = SpotifyAPI(client_id, client_secret)

In [6]:
# 1. Get FIFA Playlist tracks through search
data = client.search('https://api.spotify.com/v1/search', 'FIFA AND Soundtrack', 'playlist', '7')

Token expires: 07/19/2020, 21:37:14


In [7]:
playlist_endpoints = [] # Endpoint to list of tracks for each playlist
playlist_names = [] # Names of each playlist
playlist_num_tracks = [] # Number of tracks in each playlist
for item in data['playlists']['items']:
    playlist_endpoints.append(item['tracks']['href'])
    playlist_names.append(item['name'])
    playlist_num_tracks.append(item['tracks']['total'])

In [8]:
def fill_dict_song():
    album_name = []
    album_endpoint = []
    album_id = []

    artist1_name = []
    artist1_endpoint = []
    artist1_id = []
    
    artist2_name = []
    artist2_endpoint = []
    artist2_id = []
    
    artist3_name = []
    artist3_endpoint = []
    artist3_id = []

    track_name = []
    track_duration = []
    track_explicit = []
    track_popularity = []
    track_id = []

    for i, item in enumerate(tracks['items']):
        album_name.append(tracks['items'][i]['track']['album']['name'])
        album_endpoint.append(tracks['items'][i]['track']['album']['href'])
        album_id.append(tracks['items'][i]['track']['album']['id'])

        if len(tracks['items'][i]['track']['album']['artists']) == 1:
            artist1_name.append(tracks['items'][i]['track']['album']['artists'][0]['name'])
            artist1_endpoint.append(tracks['items'][i]['track']['album']['artists'][0]['href'])
            artist1_id.append(tracks['items'][i]['track']['album']['artists'][0]['id'])
            artist2_name.append(None)
            artist2_endpoint.append(None)
            artist2_id.append(None)
            artist3_name.append(None)
            artist3_endpoint.append(None)
            artist3_id.append(None)
        elif len(tracks['items'][i]['track']['album']['artists']) == 2:
            artist1_name.append(tracks['items'][i]['track']['album']['artists'][0]['name'])
            artist1_endpoint.append(tracks['items'][i]['track']['album']['artists'][0]['href'])
            artist1_id.append(tracks['items'][i]['track']['album']['artists'][0]['id'])
            artist2_name.append(tracks['items'][i]['track']['album']['artists'][1]['name'])
            artist2_endpoint.append(tracks['items'][i]['track']['album']['artists'][1]['href'])
            artist2_id.append(tracks['items'][i]['track']['album']['artists'][1]['id'])
            artist3_name.append(None)
            artist3_endpoint.append(None)
            artist3_id.append(None)
        elif len(tracks['items'][i]['track']['album']['artists']) == 3:
            artist1_name.append(tracks['items'][i]['track']['album']['artists'][0]['name'])
            artist1_endpoint.append(tracks['items'][i]['track']['album']['artists'][0]['href'])
            artist1_id.append(tracks['items'][i]['track']['album']['artists'][0]['id'])
            artist2_name.append(tracks['items'][i]['track']['album']['artists'][1]['name'])
            artist2_endpoint.append(tracks['items'][i]['track']['album']['artists'][1]['href'])
            artist2_id.append(tracks['items'][i]['track']['album']['artists'][1]['id'])
            artist3_name.append(tracks['items'][i]['track']['album']['artists'][2]['name'])
            artist3_endpoint.append(tracks['items'][i]['track']['album']['artists'][2]['href'])
            artist3_id.append(tracks['items'][i]['track']['album']['artists'][2]['id'])
        
        track_name.append(tracks['items'][i]['track']['name'])
        track_duration.append(tracks['items'][i]['track']['duration_ms'])
        track_explicit.append(tracks['items'][i]['track']['explicit'])
        track_popularity.append(tracks['items'][i]['track']['popularity'])
        track_id.append(tracks['items'][i]['track']['id'])
    
    dict_temp = {
        'track_name': track_name, 'track_duration': track_duration, 'track_explicit': track_explicit, 
        'track_popularity': track_popularity, 'track_id': track_id,
        'album_name': album_name, 'album_endpoint': album_endpoint, 'album_id': album_id, 
        'artist1_name': artist1_name, 'artist1_endpoint': artist1_endpoint, 'artist1_id': artist1_id,
        'artist2_name': artist2_name, 'artist2_endpoint': artist2_endpoint, 'artist2_id': artist2_id,
        'artist3_name': artist3_name, 'artist3_endpoint': artist3_endpoint, 'artist3_id': artist3_id
    }
    df_temp = pd.DataFrame(dict_temp)
    return df_temp

In [9]:
# 2. Get basic data on the tracks
dict_of_playlists = {}
for i, playlist_endpoint in enumerate(playlist_endpoints):
    tracks = client.get_resource(playlist_endpoint)
    dataframe = fill_dict_song()
    playlist_name = [playlist_names[i]]*len(dataframe)
    playlist_num_track = [playlist_num_tracks[i]]*len(dataframe)
    # Playlist Information
    dataframe.insert(0, 'num_tracks', playlist_num_track, True)
    dataframe.insert(0, 'playlist_name', playlist_name, True)
    # Add dataframe to dict
    dict_of_playlists[playlist_names[i]] = dataframe

Token expires: 07/19/2020, 21:37:15
Token expires: 07/19/2020, 21:37:15
Token expires: 07/19/2020, 21:37:15
Token expires: 07/19/2020, 21:37:16
Token expires: 07/19/2020, 21:37:16
Token expires: 07/19/2020, 21:37:16
Token expires: 07/19/2020, 21:37:17


In [10]:
dict_of_playlists['FIFA 20 Soundtrack']

,playlist_name,num_tracks,track_name,track_duration,track_explicit,track_popularity,track_id,album_name,album_endpoint,album_id,artist1_name,artist1_endpoint,artist1_id,artist2_name,artist2_endpoint,artist2_id,artist3_name,artist3_endpoint,artist3_id
0,FIFA 20 Soundtrack,41,Que Calor (feat. J Balvin & El Alfa),169997,False,73,1RKZvaLj3UPhGjZkaIrFm7,Que Calor (feat. J Balvin & El Alfa),https://api.spotify.com/v1/albums/5AKSUA3pizho...,5AKSUA3pizhoVgfjW1ijVC,Major Lazer,https://api.spotify.com/v1/artists/738wLrAtLtC...,738wLrAtLtCtFOLvQBXOXp,J Balvin,https://api.spotify.com/v1/artists/1vyhD5VmyZ7...,1vyhD5VmyZ7KMfW5gqLgo5,El Alfa,https://api.spotify.com/v1/artists/2oQX8QiMXOy...,2oQX8QiMXOyuqbcZEFsZfm
1,FIFA 20 Soundtrack,41,"Yo x Ti, Tu x Mi",201040,False,80,7BlBVFwvbWvcwNcUarQmjk,"Yo x Ti, Tu x Mi",https://api.spotify.com/v1/albums/3844bY26oeSk...,3844bY26oeSkqd06th4EYp,ROSALÍA,https://api.spotify.com/v1/artists/7ltDVBr6mKb...,7ltDVBr6mKbRvohxheJ9h1,Ozuna,https://api.spotify.com/v1/artists/1i8SpTcr7yv...,1i8SpTcr7yvPOmcqrbnVXY,None,None,None
2,FIFA 20 Soundtrack,41,Swing,179658,False,19,6T0PSfzdg8PE44T8PbbEmM,Swing,https://api.spotify.com/v1/albums/23BlliKjsR6e...,23BlliKjsR6eGFuBit4R7K,Sofi Tukker,https://api.spotify.com/v1/artists/586uxXMyD5O...,586uxXMyD5ObPuzjtrzO1Q,None,None,None,None,None,None
3,FIFA 20 Soundtrack,41,Vámono,208329,False,46,623mI2F1iCO4cqGbv6WUVn,Vámono,https://api.spotify.com/v1/albums/0DAtzeZ09MxU...,0DAtzeZ09MxUORBOwGWxF5,Buscabulla,https://api.spotify.com/v1/artists/0MoaBi6dSqu...,0MoaBi6dSquXp6rrlqlF8R,None,None,None,None,None,None
4,FIFA 20 Soundtrack,41,Fado,250400,False,42,3YT3RgSGhvgormc5f7h2xi,Fado,https://api.spotify.com/v1/albums/1ZZwX6IB8WGD...,1ZZwX6IB8WGDPB314R8yXl,Milky Chance,https://api.spotify.com/v1/artists/1hzfo8twXdO...,1hzfo8twXdOegF3xireCYs,None,None,None,None,None,None
5,FIFA 20 Soundtrack,41,Offence,168521,True,54,4cakdyTV2GRb9ktqBNMChH,GREY Area,https://api.spotify.com/v1/albums/4Wwm4xg2748z...,4Wwm4xg2748zhYuzDRFTgY,Little Simz,https://api.spotify.com/v1/artists/6eXZu6O7nAU...,6eXZu6O7nAUA5z6vLV8NKI,None,None,None,None,None,None
6,FIFA 20 Soundtrack,41,Unemployed,168561,False,36,4Qz6dwfC5VRaOPBw9ZbWzM,Unemployed,https://api.spotify.com/v1/albums/4wT85WYKlyU9...,4wT85WYKlyU9XZddZAffSq,Tierra Whack,https://api.spotify.com/v1/artists/4lPl9gqgox3...,4lPl9gqgox3JDiaJ1yklKh,None,None,None,None,None,None
7,FIFA 20 Soundtrack,41,RUNAWAY,161759,False,29,6iRXOuVt9bOdGr3nohzddj,RUNAWAY,https://api.spotify.com/v1/albums/3bdA73eL6cN5...,3bdA73eL6cN51aC2rPFfyc,half•alive,https://api.spotify.com/v1/artists/7sOR7gk6XUl...,7sOR7gk6XUlGnxj3p9F54k,None,None,None,None,None,None
8,FIFA 20 Soundtrack,41,Ozone,197248,True,47,6mOHAShVtqwJIBRAd7Xa4l,Ozone,https://api.spotify.com/v1/albums/4x9Nh0dyy93t...,4x9Nh0dyy93tsBOQylu5pM,JyellowL,https://api.spotify.com/v1/artists/7zWrMTItje9...,7zWrMTItje9ZjKK2XWTPle,None,None,None,None,None,None
9,FIFA 20 Soundtrack,41,Frens,198849,False,41,15FfKpV7MURQncDusPvqwM,Frens,https://api.spotify.com/v1/albums/6ptrrZitJ1n2...,6ptrrZitJ1n2a6qraQGmLz,Obongjayar,https://api.spotify.com/v1/artists/6l7R1jntPah...,6l7R1jntPahGxwJt7Tky8h,None,None,None,None,None,None


In [27]:
# 3. Get audio data of each of the tracks
track_ids = []
for key, value in dict_of_playlists.items(): #get list of song ids
    track_id_list = value['track_id'].tolist()
    track_ids.append(track_id_list)

In [28]:
track_ids = list(np.concatenate(track_ids).flat) #turn into 1d list

In [29]:
len(track_ids) #can only take 100 at a time

294

In [34]:
audio_feature_data = client.get_audio_features(track_ids[:100])

Token expires: 07/19/2020, 21:45:36


In [35]:
audio_feature_data

{'audio_features': [{'danceability': 0.795,
   'energy': 0.885,
   'key': 8,
   'loudness': -4.13,
   'mode': 1,
   'speechiness': 0.0713,
   'acousticness': 0.0375,
   'instrumentalness': 0.00638,
   'liveness': 0.292,
   'valence': 0.46,
   'tempo': 126.037,
   'type': 'audio_features',
   'id': '1RKZvaLj3UPhGjZkaIrFm7',
   'uri': 'spotify:track:1RKZvaLj3UPhGjZkaIrFm7',
   'track_href': 'https://api.spotify.com/v1/tracks/1RKZvaLj3UPhGjZkaIrFm7',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1RKZvaLj3UPhGjZkaIrFm7',
   'duration_ms': 169998,
   'time_signature': 4},
  {'danceability': 0.78,
   'energy': 0.759,
   'key': 1,
   'loudness': -3.292,
   'mode': 1,
   'speechiness': 0.225,
   'acousticness': 0.462,
   'instrumentalness': 0.000868,
   'liveness': 0.143,
   'valence': 0.579,
   'tempo': 91.952,
   'type': 'audio_features',
   'id': '7BlBVFwvbWvcwNcUarQmjk',
   'uri': 'spotify:track:7BlBVFwvbWvcwNcUarQmjk',
   'track_href': 'https://api.spotify.com/v1/tracks/7B

In [ ]:
audio_feature_track_id = []
acousticness = []
danceability = []
energy = []
instrumentalness = []
key = []
loudness = []
mode = []



In [12]:
# 4. Get data on the artists of the tracks

In [13]:
# 5. Get data on the albums the tracks are in